In [2]:
import pandas as pd 
import numpy as np

In [3]:
# Read data ---------------------------------------------------------------
cig_data = pd.read_csv("/Users/ellenwu/homework3_attempt1-/data/input/The_Tax_Burden_on_Tobacco__1970-2019_20250304.csv")
cpi_data = pd.read_excel("/Users/ellenwu/homework3_attempt1-/data/input/SeriesReport-20250304165118_4f02a8.xlsx", skiprows=11)

/Users/ellenwu/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
# Check which years and measures exist
print(cig_data[['Year', 'SubMeasureDesc']].drop_duplicates().sort_values(by='Year'))

      Year                                     SubMeasureDesc
293   1970  Federal and State tax as a Percentage of Retai...
1110  1970                              Average Cost per pack
71    1970                     Federal and State Tax per pack
100   1970                        Gross Cigarette Tax Revenue
382   1970                                 State Tax per pack
...    ...                                                ...
341   2019                                 State Tax per pack
681   2019                              Average Cost per pack
65    2019                     Federal and State Tax per pack
501   2019                        Gross Cigarette Tax Revenue
591   2019  Federal and State tax as a Percentage of Retai...

[300 rows x 2 columns]


In [5]:
# Clean tobacco data ------------------------------------------------------
def map_measure(desc):
    mapping = {
        "average cost per pack": "cost_per_pack",
        "cigarette consumption (pack sales per capita)": "sales_per_capita",
        "federal and state tax as a percentage of retail price": "tax_percent",
        "federal and state tax per pack": "tax_dollar",
        "gross cigarette tax revenue": "tax_revenue",
        "state tax per pack": "tax_state"
    }
    return mapping.get(desc.strip().lower(), None)

cig_data['measure'] = cig_data['SubMeasureDesc'].str.strip().str.lower().map(map_measure)

In [6]:
print(cig_data[cig_data['Year'] >= 1991]['SubMeasureDesc'].unique())

['State Tax per pack' 'Average Cost per pack'
 'Federal and State tax as a Percentage of Retail Price'
 'Gross Cigarette Tax Revenue' 'Federal and State Tax per pack'
 'Cigarette Consumption (Pack Sales Per Capita)']


In [7]:
print("Non-missing Data_Values by Year:")
print(cig_data.groupby('Year')['Data_Value'].count())

final_data = cig_data.pivot_table(index=['state', 'Year'], 
                                  columns='measure', 
                                  values='value').reset_index()

print("Years in final_data after pivot:")
print(final_data['Year'].unique())

Non-missing Data_Values by Year:
Year
1970    306
1971    306
1972    306
1973    306
1974    306
1975    306
1976    306
1977    306
1978    306
1979    306
1980    306
1981    306
1982    306
1983    306
1984    306
1985    306
1986    306
1987    306
1988    306
1989    306
1990    306
1991    306
1992    306
1993    306
1994    306
1995    306
1996    306
1997    306
1998    306
1999    306
2000    306
2001    306
2002    306
2003    306
2004    306
2005    306
2006    306
2007    306
2008    306
2009    306
2010    306
2011    306
2012    306
2013    306
2014    306
2015    306
2016    306
2017    306
2018    306
2019    306
Name: Data_Value, dtype: int64


KeyError: 'value'

In [8]:
cig_data = cig_data[['LocationAbbr', 'LocationDesc', 'Year', 'Data_Value', 'measure']].rename(
    columns={
        'LocationAbbr': 'state_abb',
        'LocationDesc': 'state',
        'Data_Value': 'value'
    }
)

final_data = cig_data.pivot_table(index=['state', 'Year'], 
                                  columns='measure', 
                                  values='value').reset_index()

print("Years present after pivot:")
print(final_data['Year'].unique())

Years present after pivot:
[1970 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1982 1983
 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997
 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011
 2012 2013 2014 2015 2016 2017 2018 2019]


In [9]:
print(final_data[['Year', 'tax_state']].groupby('Year').count())

measure  tax_state
Year              
1970            51
1971            51
1972            51
1973            51
1974            51
1975            51
1976            51
1977            51
1978            51
1979            51
1980            51
1981            51
1982            51
1983            51
1984            51
1985            51
1986            51
1987            51
1988            51
1989            51
1990            51
1991            51
1992            51
1993            51
1994            51
1995            51
1996            51
1997            51
1998            51
1999            51
2000            51
2001            51
2002            51
2003            51
2004            51
2005            51
2006            51
2007            51
2008            51
2009            51
2010            51
2011            51
2012            51
2013            51
2014            51
2015            51
2016            51
2017            51
2018            51
2019            51


In [10]:
print(final_data.groupby('state')['tax_state'].nunique())

state
Alabama                  5
Alaska                   7
Arizona                  7
Arkansas                 9
California               5
Colorado                 5
Connecticut             15
Delaware                 7
District of Columbia    16
Florida                  6
Georgia                  3
Hawaii                  30
Idaho                    5
Illinois                 7
Indiana                  5
Iowa                     8
Kansas                   6
Kentucky                 5
Louisiana                7
Maine                   11
Maryland                 8
Massachusetts            9
Michigan                 6
Minnesota               17
Mississippi              4
Missouri                 3
Montana                  7
Nebraska                 7
Nevada                   6
New Hampshire           14
New Jersey              12
New Mexico               5
New York                10
North Carolina           5
North Dakota             7
Ohio                     8
Oklahoma              

In [11]:
final_data = final_data.sort_values(by=['state', 'Year'])

In [12]:
# Clean CPI data ----------------------------------------------------------
cpi_data = cpi_data.melt(id_vars=['Year'], 
                         value_vars=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                                     'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'],
                         var_name='month', 
                         value_name='index')

cpi_data = cpi_data.groupby('Year', as_index=False).agg({'index': 'mean'})

In [14]:
print(cpi_data['Year'].unique())

[1913 1914 1915 1916 1917 1918 1919 1920 1921 1922 1923 1924 1925 1926
 1927 1928 1929 1930 1931 1932 1933 1934 1935 1936 1937 1938 1939 1940
 1941 1942 1943 1944 1945 1946 1947 1948 1949 1950 1951 1952 1953 1954
 1955 1956 1957 1958 1959 1960 1961 1962 1963 1964 1965 1966 1967 1968
 1969 1970 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1982
 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996
 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010
 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 2024
 2025]


In [16]:
# Form final dataset ------------------------------------------------------
# adjust to 2010 dollars (using 2010 CPI = 218)
final_data = final_data.merge(cpi_data, on='Year', how='left')

print("Years present in final_data after merging CPI:")
print(final_data[['Year', 'index']].drop_duplicates().sort_values('Year'))


Years present in final_data after merging CPI:
    Year       index
0   1970   38.825000
1   1971   40.491667
2   1972   41.816667
3   1973   44.400000
4   1974   49.308333
5   1975   53.816667
6   1976   56.908333
7   1977   60.608333
8   1978   65.233333
9   1979   72.575000
10  1980   82.408333
11  1981   90.925000
12  1982   96.500000
13  1983   99.600000
14  1984  103.883333
15  1985  107.566667
16  1986  109.608333
17  1987  113.625000
18  1988  118.258333
19  1989  123.966667
20  1990  130.658333
21  1991  136.191667
22  1992  140.316667
23  1993  144.458333
24  1994  148.225000
25  1995  152.383333
26  1996  156.850000
27  1997  160.516667
28  1998  163.008333
29  1999  166.575000
30  2000  172.200000
31  2001  177.066667
32  2002  179.875000
33  2003  183.958333
34  2004  188.883333
35  2005  195.291667
36  2006  201.591667
37  2007  207.342417
38  2008  215.302500
39  2009  214.537000
40  2010  218.055500
41  2011  224.939167
42  2012  229.593917
43  2013  232.957083
44  2014

In [17]:
final_data['tax_change'] = final_data.groupby('state')['tax_state'].diff().notna()
print(final_data[['state', 'Year', 'tax_state', 'tax_change']].tail(50))

        state  Year  tax_state  tax_change
2500  Wyoming  1970       0.06       False
2501  Wyoming  1971       0.08        True
2502  Wyoming  1972       0.08        True
2503  Wyoming  1973       0.08        True
2504  Wyoming  1974       0.08        True
2505  Wyoming  1975       0.08        True
2506  Wyoming  1976       0.08        True
2507  Wyoming  1977       0.08        True
2508  Wyoming  1978       0.08        True
2509  Wyoming  1979       0.08        True
2510  Wyoming  1980       0.08        True
2511  Wyoming  1981       0.08        True
2512  Wyoming  1982       0.08        True
2513  Wyoming  1983       0.08        True
2514  Wyoming  1984       0.08        True
2515  Wyoming  1985       0.08        True
2516  Wyoming  1986       0.08        True
2517  Wyoming  1987       0.08        True
2518  Wyoming  1988       0.08        True
2519  Wyoming  1989       0.08        True
2520  Wyoming  1990       0.08        True
2521  Wyoming  1991       0.12        True
2522  Wyomi

In [19]:
final_data['price_cpi'] = final_data['cost_per_pack'] * (218 / final_data['index'])

In [20]:
# Write output files ------------------------------------------------------
final_data.to_csv("/Users/ellenwu/homework3_attempt1-/data/output/dTaxBurden_Data.txt", sep='\t', index=False)
final_data.to_csv("/Users/ellenwu/homework3_attempt1-/data/output/dTaxBurden_Data.csv", index=False)
